 # price predictor


In [1]:
import pandas as pd


In [2]:
housing=pd.read_csv("data.csv")

In [3]:
housing.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,NaN,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


# splitting the data into training and testing set

In [4]:
   from sklearn.model_selection import train_test_split

In [5]:
train_set,test_set=train_test_split(housing,test_size=0.2,random_state=42)

In [6]:
print(f"rows in train set:{len(train_set)}\nrows in test set:{len(test_set)}\n")

rows in train set:404
rows in test set:102



# stratifying the data

In [7]:
from sklearn.model_selection import StratifiedShuffleSplit
split=StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index,test_index in split.split(housing,housing['CHAS']):
    strat_train_set=housing.loc[train_index]
    strat_test_set=housing.loc[test_index]

In [8]:
strat_test_set['CHAS'].value_counts()

0    95
1     7
Name: CHAS, dtype: int64

In [9]:
strat_train_set['CHAS'].value_counts()

0    376
1     28
Name: CHAS, dtype: int64

In [10]:
95/7


13.571428571428571

In [11]:
376/28

13.428571428571429

In [12]:
housing=strat_train_set.copy()

# looking for correlations

In [13]:
#correlation of every value with respect with medv that is price
#is found out 

In [14]:
corr_matrix=housing.corr()
corr_matrix['MEDV'].sort_values(ascending=False)

MEDV       1.000000
RM         0.679273
B          0.361761
ZN         0.339741
DIS        0.240451
CHAS       0.205066
AGE       -0.364596
RAD       -0.374693
CRIM      -0.393715
NOX       -0.422873
TAX       -0.456657
INDUS     -0.473516
PTRATIO   -0.493534
LSTAT     -0.740494
Name: MEDV, dtype: float64

# # making a new attribute by combining two old attributes

In [15]:
housing['TAXRM']=housing['TAX']/housing['RM']

In [16]:
# corelation wrt txrm is also made

In [17]:
corr_matrix=housing.corr()
corr_matrix['MEDV'].sort_values(ascending=False)

MEDV       1.000000
RM         0.679273
B          0.361761
ZN         0.339741
DIS        0.240451
CHAS       0.205066
AGE       -0.364596
RAD       -0.374693
CRIM      -0.393715
NOX       -0.422873
TAX       -0.456657
INDUS     -0.473516
PTRATIO   -0.493534
TAXRM     -0.524020
LSTAT     -0.740494
Name: MEDV, dtype: float64

In [18]:
housing.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,TAXRM
count,404.000000,404.000000,404.000000,404.000000,404.000000,401.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,401.000000
mean,3.602814,10.836634,11.344950,0.069307,0.558064,6.279062,69.039851,3.746210,9.735149,412.341584,18.473267,353.392822,12.791609,22.509406,67.651861
std,8.099383,22.150636,6.877817,0.254290,0.116875,0.714601,28.258248,2.099057,8.731259,168.672623,2.129243,96.069235,7.235740,9.385531,31.503002
min,0.006320,0.000000,0.740000,0.000000,0.389000,3.561000,2.900000,1.129600,1.000000,187.000000,13.000000,0.320000,1.730000,5.000000,24.645639
25%,0.086962,0.000000,5.190000,0.000000,0.453000,5.878000,44.850000,2.035975,4.000000,284.000000,17.400000,374.617500,6.847500,16.600000,44.807506
50%,0.286735,0.000000,9.900000,0.000000,0.538000,6.211000,78.200000,3.122200,5.000000,337.000000,19.000000,390.955000,11.570000,21.150000,54.831220
75%,3.731923,12.500000,18.100000,0.000000,0.631000,6.630000,94.100000,5.100400,24.000000,666.000000,20.200000,395.630000,17.102500,25.000000,98.666667
max,73.534100,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,36.980000,50.000000,187.026116


In [19]:
#t  he values of rm were 401 since three data points were missing

In [20]:
housing=strat_train_set.drop("MEDV",axis=1)
housing_labels=strat_train_set["MEDV"].copy()

# Handling missing attributes through imputer class of sklearn

In [21]:
#the median will be calculated for each colomn of data set and
#we will create our pipeline in such a way that missing points 
#in each clomn
#will be filled with corresponding median values of that column

In [22]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy="median")
imputer.fit(housing)

SimpleImputer(strategy='median')

In [23]:
imputer.statistics_

array([2.86735e-01, 0.00000e+00, 9.90000e+00, 0.00000e+00, 5.38000e-01,
       6.21100e+00, 7.82000e+01, 3.12220e+00, 5.00000e+00, 3.37000e+02,
       1.90000e+01, 3.90955e+02, 1.15700e+01])

In [24]:
imputer.statistics_.shape

(13,)

# filling the missing points

In [25]:
X=imputer.transform(housing)
housing_tr=pd.DataFrame(X,columns=housing.columns)
housing_tr.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000
mean,3.602814,10.836634,11.344950,0.069307,0.558064,6.278557,69.039851,3.746210,9.735149,412.341584,18.473267,353.392822,12.791609
std,8.099383,22.150636,6.877817,0.254290,0.116875,0.711961,28.258248,2.099057,8.731259,168.672623,2.129243,96.069235,7.235740
min,0.006320,0.000000,0.740000,0.000000,0.389000,3.561000,2.900000,1.129600,1.000000,187.000000,13.000000,0.320000,1.730000
25%,0.086962,0.000000,5.190000,0.000000,0.453000,5.878750,44.850000,2.035975,4.000000,284.000000,17.400000,374.617500,6.847500
50%,0.286735,0.000000,9.900000,0.000000,0.538000,6.211000,78.200000,3.122200,5.000000,337.000000,19.000000,390.955000,11.570000
75%,3.731923,12.500000,18.100000,0.000000,0.631000,6.630000,94.100000,5.100400,24.000000,666.000000,20.200000,395.630000,17.102500
max,73.534100,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,36.980000


In [26]:
#after imputing the data the values of rm became 404 since 
#three missing points
#were filled by the median value of rm

# feature scaling

In [27]:
#we will use standardScaler class of sklearn to scale down 
#the parameters of all the features

# making a pipeline

In [28]:
#import sklearn

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
my_pipeline=Pipeline([('imputer',SimpleImputer(strategy="median")),
                     ('std_scaler',StandardScaler()),
                     ])

In [30]:
housing_num_tr=my_pipeline.fit_transform(housing_tr)

In [31]:
housing_num_tr

array([[-0.43942006,  3.12628155, -1.12165014, ..., -0.97491834,
         0.41164221, -0.86091034],
       [-0.44352175,  3.12628155, -1.35893781, ..., -0.69277865,
         0.39131918, -0.94116739],
       [ 0.15682292, -0.4898311 ,  0.98336806, ...,  0.81196637,
         0.44624347,  0.81480158],
       ...,
       [-0.43525657, -0.4898311 , -1.23083158, ..., -0.22254583,
         0.41831233, -1.27603303],
       [ 0.14210728, -0.4898311 ,  0.98336806, ...,  0.81196637,
        -3.15239177,  0.73869575],
       [-0.43974024, -0.4898311 ,  0.37049623, ..., -0.97491834,
         0.41070422,  0.09940681]])

In [32]:
housing_num_tr.shape

(404, 13)

# selecting a desired model

In [33]:
#selecting first model

from sklearn.linear_model import LinearRegression
 model=LinearRegression()


#selectiong 2nd  model

from sklearn.tree import DecisionTreeRegressor
#model=DecisionTreeRegressor()


#selecting 3rd  model

from sklearn.ensemble import RandomForestRegressor
#model=RandomForestRegressor()

#fitting the data

model.fit(housing_num_tr,housing_labels)

RandomForestRegressor()

# analysing some part of our data

In [34]:
some_data= housing.iloc[:5]

In [35]:
some_labels=housing_labels.iloc[:5]

In [36]:
prepared_data=my_pipeline.transform(some_data)

In [37]:
model.predict(prepared_data)

array([22.262, 25.58 , 16.316, 23.385, 23.659])

In [38]:
list(some_labels)

[21.9, 24.5, 16.7, 23.1, 23.0]

# Evaluating our model by finding out the error rate

In [39]:
import numpy as np

In [40]:
from sklearn.metrics import mean_squared_error
housing_predictions=model.predict(housing_num_tr)
mse=mean_squared_error(housing_labels,housing_predictions)
rmse=np.sqrt(mse)

In [41]:
rmse

1.219185137095613

# using better evaluation by cross validations

In [42]:
from sklearn.model_selection import cross_val_score
scores=cross_val_score(model,housing_num_tr,housing_labels,scoring="neg_mean_squared_error",cv=10)
rmse_scores=np.sqrt(-scores)

In [43]:
rmse_scores

array([2.70139132, 2.74799319, 4.65319933, 2.71870657, 3.33406253,
       2.78306881, 4.42612215, 3.30418868, 3.17141223, 3.13817206])

In [44]:
#since we had divided our data in 10 parts therefore error for 
#all 10 set were found out

# finding out mean and standard deviation for these 10 set scores

In [45]:
def print_scores(scores):
    print("scores:",scores)
    print("mean:",scores.mean())
    print("standard deviation:",scores.std())

In [46]:
print_scores(rmse_scores)

scores: [2.70139132 2.74799319 4.65319933 2.71870657 3.33406253 2.78306881
 4.42612215 3.30418868 3.17141223 3.13817206]
mean: 3.2978316875782907
standard deviation: 0.6641774764485725


In [47]:
#this model RandomForestRegressor performed the best and hence
#it was selected

# saving the model

using the joblib and dumping the model in joblib file

In [48]:
from joblib import dump,load
dump(model,'Dragon.joblib')

['Dragon.joblib']

# testing the model on test data

In [49]:
X_test=strat_test_set.drop("MEDV",axis=1)
Y_test=strat_test_set["MEDV"].copy()
X_test_prepared=my_pipeline.transform(X_test)
final_predictions=model.predict(X_test_prepared)
final_mse=mean_squared_error(Y_test,final_predictions)
final_rmse=np.sqrt(final_mse)


In [50]:
final_rmse

2.912880570103462

# other works 

In [52]:
prepared_data[0]

array([-0.43942006,  3.12628155, -1.12165014, -0.27288841, -1.42262747,
       -0.23985651, -1.31238772,  2.61111401, -1.0016859 , -0.5778192 ,
       -0.97491834,  0.41164221, -0.86091034])

# usage of model 

In [54]:
from joblib import dump,load
import numpy as np
#we loaded our model stored in dragon.joblib file 
model=load('Dragon.joblib')

#we gave the values of all 14 features and stored them in an array
features=np.array([[-0.43942006,  3.12628155, -1.12165014, -0.27288841, -1.42262747,
       -0.23985651, -1.31238772,  2.61111401, -1.0016859 , -0.5778192 ,
       -0.97491834,  0.41164221, -0.86091034]])

#we passed those values to predict the price of our house
model.predict(features)

array([22.262])